In [22]:
!pip install qiskit qiskit_aer

In [23]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

In [24]:
def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

In [25]:
def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

In [26]:
def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

In [27]:
def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    # Create a simulator and save the statevector explicitly
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    # Transpile and run
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()

    # Retrieve statevector safely
    statevector = result.data(0)["statevector"]

    # Display results
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))

if __name__ == "__main__":
    n_qubits = 3
    print(f"Running Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      


# **task**

In [28]:
pip install qiskit qiskit-aer


In [29]:
# Overwrite qft_tasks.py with all missing functions included

complete_qft_tasks = r'''
import math
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile

try:
    from qiskit_aer import Aer
except Exception:
    from qiskit import Aer


# -------------------- QFT --------------------
def qft(n: int) -> QuantumCircuit:
    """Quantum Fourier Transform on n qubits."""
    qc = QuantumCircuit(n, name=f"QFT_{n}")
    for j in range(n):
        qc.h(j)
        for k in range(j + 1, n):
            angle = math.pi / (2 ** (k - j))
            qc.cp(angle, k, j)
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
    return qc


# -------------------- Inverse QFT --------------------
def iqft(n: int) -> QuantumCircuit:
    """Inverse Quantum Fourier Transform."""
    qc = QuantumCircuit(n, name=f"QFT†_{n}")
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
    for j in reversed(range(n)):
        for k in reversed(range(j + 1, n)):
            angle = -math.pi / (2 ** (k - j))
            qc.cp(angle, k, j)
        qc.h(j)
    return qc


# -------------------- Controlled-U --------------------
def controlled_U_power(phi: float, power: int) -> QuantumCircuit:
    """Creates U^(2^power), where U = Phase(2πφ) on |1>."""
    theta = 2 * math.pi * phi * (2 ** power)
    base = QuantumCircuit(1, name=f"U^{2**power}")
    base.p(theta, 0)
    return base


# -------------------- QPE --------------------
def build_simple_qpe(phi: float, m: int = 3) -> QuantumCircuit:
    """Builds a simple Quantum Phase Estimation (QPE) circuit."""
    cnt = QuantumRegister(m, "count")
    tgt = QuantumRegister(1, "tgt")
    c = ClassicalRegister(m, "c")
    qc = QuantumCircuit(cnt, tgt, c, name=f"QPE_phi_{phi:.3f}")

    qc.x(tgt[0])  # prepare eigenstate |1>
    for j in range(m):
        qc.h(cnt[j])

    # Controlled-U^(2^k)
    for j in range(m):
        Uj = controlled_U_power(phi, m - 1 - j)
        qc.append(Uj.to_gate().control(1), [cnt[j], tgt[0]])

    qc.append(iqft(m), cnt[:])
    qc.measure(cnt, c)
    return qc


# -------------------- Helpers --------------------
def simulate_counts(qc: QuantumCircuit, shots: int = 2048):
    """Simulate a circuit and return measurement counts."""
    try:
        backend = Aer.get_backend("aer_simulator")
    except Exception:
        backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    result = job.result()
    try:
        return result.get_counts()
    except Exception:
        return result.get_counts(tqc)


def draw_and_show(qc: QuantumCircuit, title=None, style="mpl"):
    """Draw a circuit using matplotlib or text mode."""
    if style == "text":
        print(qc.draw("text"))
    else:
        fig = qc.draw("mpl")
        if title:
            plt.title(title)
        plt.show()
'''

with open("qft_tasks.py", "w") as f:
    f.write(complete_qft_tasks)

print("✅ qft_tasks.py rebuilt successfully with all functions.")


✅ qft_tasks.py rebuilt successfully with all functions.


In [30]:
import math
from qiskit import QuantumCircuit

def qft(n: int) -> QuantumCircuit:
    """Quantum Fourier Transform on n qubits."""
    qc = QuantumCircuit(n, name=f"QFT_{n}")
    for j in range(n):
        qc.h(j)
        for k in range(j + 1, n):
            angle = math.pi / (2 ** (k - j))
            qc.cp(angle, k, j)
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
    return qc


def iqft(n: int) -> QuantumCircuit:
    """Inverse Quantum Fourier Transform."""
    qc = QuantumCircuit(n, name=f"QFT†_{n}")
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
    for j in reversed(range(n)):
        for k in reversed(range(j + 1, n)):
            angle = -math.pi / (2 ** (k - j))
            qc.cp(angle, k, j)
        qc.h(j)
    return qc


In [31]:
import importlib, qft_tasks

# Force reload the module so Python picks up the new definitions
importlib.reload(qft_tasks)

# Check which functions exist
print("Functions available in qft_tasks.py:")
print([f for f in dir(qft_tasks) if not f.startswith("__")])


Functions available in qft_tasks.py:
['Aer', 'ClassicalRegister', 'QuantumCircuit', 'QuantumRegister', 'build_simple_qpe', 'controlled_U_power', 'draw_and_show', 'iqft', 'math', 'plt', 'qft', 'simulate_counts', 'transpile']


In [32]:
import matplotlib.pyplot as plt

def draw_and_show(qc, title=None, style="mpl"):
    """Draw a Qiskit circuit as text or matplotlib diagram."""
    if style == "text":
        print(qc.draw("text"))
    else:
        fig = qc.draw("mpl")
        if title:
            plt.title(title)
        plt.show()

def simulate_counts(qc, shots=1024):
    """Simulate a Qiskit circuit and return measurement counts."""
    from qiskit import transpile
    try:
        from qiskit_aer import Aer
    except Exception:
        from qiskit import Aer

    try:
        backend = Aer.get_backend("aer_simulator")
    except Exception:
        backend = Aer.get_backend("qasm_simulator")

    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    result = job.result()
    try:
        return result.get_counts()
    except Exception:
        return result.get_counts(tqc)


# **Vary** number of qubits (2, 3, 4) and visualize circuits

In [33]:
from qft_tasks import qft # Import the qft function

for n in [2, 3, 4]:
    qc = qft(n)
    print(qc.draw("text"))

     ┌───┐                 
q_0: ┤ H ├─■─────────────X─
     └───┘ │P(π/2) ┌───┐ │ 
q_1: ──────■───────┤ H ├─X─
                   └───┘   
     ┌───┐                                        
q_0: ┤ H ├─■────────■───────────────────────────X─
     └───┘ │P(π/2)  │       ┌───┐               │ 
q_1: ──────■────────┼───────┤ H ├─■─────────────┼─
                    │P(π/4) └───┘ │P(π/2) ┌───┐ │ 
q_2: ───────────────■─────────────■───────┤ H ├─X─
                                          └───┘   
     ┌───┐                                                                     »
q_0: ┤ H ├─■────────■─────────────■────────────────────────────────────────────»
     └───┘ │P(π/2)  │       ┌───┐ │                                            »
q_1: ──────■────────┼───────┤ H ├─┼────────■────────■───────────────────────X──»
                    │P(π/4) └───┘ │        │P(π/2)  │       ┌───┐           │  »
q_2: ───────────────■─────────────┼────────■────────┼───────┤ H ├─■─────────X──»
                 

In [34]:
pip install qiskit qiskit-aer


# **Inverse** QFT and verification (QFT followed by IQFT)

In [35]:
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

for n in [2, 3, 4]:
    qc_test = QuantumCircuit(n, n)
    qc_test.x(n - 1)  # prepare a non-trivial basis state |...1>
    qc_test.append(qft(n), range(n))
    qc_test.append(iqft(n), range(n))
    qc_test.barrier()
    qc_test.measure(range(n), range(n))

    draw_and_show(qc_test, title=f"Verify QFT†∘QFT on n={n}", style="text")

    counts = simulate_counts(qc_test, shots=1024)
    plot_histogram(counts, title=f"Recovered after QFT†∘QFT (n={n})")
    plt.show()


          ┌────────┐┌─────────┐ ░ ┌─┐   
q_0: ─────┤0       ├┤0        ├─░─┤M├───
     ┌───┐│  QFT_2 ││  QFT†_2 │ ░ └╥┘┌─┐
q_1: ┤ X ├┤1       ├┤1        ├─░──╫─┤M├
     └───┘└────────┘└─────────┘ ░  ║ └╥┘
c: 2/══════════════════════════════╩══╩═
                                   0  1 
          ┌────────┐┌─────────┐ ░ ┌─┐      
q_0: ─────┤0       ├┤0        ├─░─┤M├──────
          │        ││         │ ░ └╥┘┌─┐   
q_1: ─────┤1 QFT_3 ├┤1 QFT†_3 ├─░──╫─┤M├───
     ┌───┐│        ││         │ ░  ║ └╥┘┌─┐
q_2: ┤ X ├┤2       ├┤2        ├─░──╫──╫─┤M├
     └───┘└────────┘└─────────┘ ░  ║  ║ └╥┘
c: 3/══════════════════════════════╩══╩══╩═
                                   0  1  2 
          ┌────────┐┌─────────┐ ░ ┌─┐         
q_0: ─────┤0       ├┤0        ├─░─┤M├─────────
          │        ││         │ ░ └╥┘┌─┐      
q_1: ─────┤1       ├┤1        ├─░──╫─┤M├──────
          │  QFT_4 ││  QFT†_4 │ ░  ║ └╥┘┌─┐   
q_2: ─────┤2       ├┤2        ├─░──╫──╫─┤M├───
     ┌───┐│        ││         │ ░  

# **Phase** Estimation (φ = 1/8) with m = 2, 3, 4 counting qubits

In [36]:
from qft_tasks import build_simple_qpe, draw_and_show, simulate_counts
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

phi = 1 / 8

for m in [2, 3, 4]:
    qpe_circ = build_simple_qpe(phi, m=m)
    draw_and_show(qpe_circ, title=f"QPE circuit (m={m}, phi={phi})", style="text")
    counts = simulate_counts(qpe_circ, shots=4096)
    expected = format(int(phi * (2 ** m)) % (2 ** m), f"0{m}b")
    plot_histogram(counts, title=f"QPE results m={m}, expected ≈ {expected}")
    plt.show()


         ┌───┐              ┌─────────┐┌─┐   
count_0: ┤ H ├───■──────────┤0        ├┤M├───
         ├───┤   │          │  QFT†_2 │└╥┘┌─┐
count_1: ┤ H ├───┼──────■───┤1        ├─╫─┤M├
         ├───┤┌──┴──┐┌──┴──┐└─────────┘ ║ └╥┘
    tgt: ┤ X ├┤ U^2 ├┤ U^1 ├────────────╫──╫─
         └───┘└─────┘└─────┘            ║  ║ 
    c: 2/═══════════════════════════════╩══╩═
                                        0  1 
         ┌───┐                     ┌─────────┐┌─┐      
count_0: ┤ H ├───■─────────────────┤0        ├┤M├──────
         ├───┤   │                 │         │└╥┘┌─┐   
count_1: ┤ H ├───┼──────■──────────┤1 QFT†_3 ├─╫─┤M├───
         ├───┤   │      │          │         │ ║ └╥┘┌─┐
count_2: ┤ H ├───┼──────┼──────■───┤2        ├─╫──╫─┤M├
         ├───┤┌──┴──┐┌──┴──┐┌──┴──┐└─────────┘ ║  ║ └╥┘
    tgt: ┤ X ├┤ U^4 ├┤ U^2 ├┤ U^1 ├────────────╫──╫──╫─
         └───┘└─────┘└─────┘└─────┘            ║  ║  ║ 
    c: 3/══════════════════════════════════════╩══╩══╩═
                          

# **bold text** Measure output states and show probability distribution

In [37]:
from qiskit import QuantumCircuit
from qft_tasks import qft, draw_and_show, simulate_counts # Import necessary functions
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

n = 4
qc_measure = QuantumCircuit(n, n)
qc_measure.x(0); qc_measure.x(1); qc_measure.x(2)  # prepare |0111>
qc_measure.append(qft(n), range(n))
qc_measure.measure(range(n), range(n))
draw_and_show(qc_measure, title="Measure distribution after QFT on |0111> (n=4)", style="text")
counts = simulate_counts(qc_measure, shots=4096)
plot_histogram(counts, title="Measurement distribution after QFT(|0111>)")
plt.show()

     ┌───┐┌────────┐┌─┐         
q_0: ┤ X ├┤0       ├┤M├─────────
     ├───┤│        │└╥┘┌─┐      
q_1: ┤ X ├┤1       ├─╫─┤M├──────
     ├───┤│  QFT_4 │ ║ └╥┘┌─┐   
q_2: ┤ X ├┤2       ├─╫──╫─┤M├───
     └───┘│        │ ║  ║ └╥┘┌─┐
q_3: ─────┤3       ├─╫──╫──╫─┤M├
          └────────┘ ║  ║  ║ └╥┘
c: 4/════════════════╩══╩══╩══╩═
                     0  1  2  3 
